In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
def dydx(x,y):
    
    #set derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we wil set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs


In [4]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi + .5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derives = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temps[:] = yi[:] + .5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temps[:] = yi[:] + .5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temps[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

In [6]:
def rk4_mv_ad(dydx,xi,yi,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,t*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol>1.0):
        
        #estimate our error by taking one step of size h
        #vs. two steps of the same size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,.5*h_step)
        y_11 = rk4_mv_core(dydx, x_i+.5*h_step, y_i, nv, .5*h_step)

        #compute the error
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step size
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i += 1
        
    #next time, try to take a larger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step


In [7]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are boudary conditions
    #tol is the tolerance for integrting y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #break
        flag = 0
        
    #update values
    xi += h_step
    yi[:] = yi_new[:]
    
    #add the step to the arrays
    x = np.append(x,xi)
    y_new = np.zeros((len(x),nv))
    y_new[0:len(x)-1,:] = y
    y_new[-1,:] = yi[:]
    del y
    y = y_new
    
    #prevent too many iterations
    if(i>=imax):
        
        print("maximum iterations reached.")
        raise StopIteration("Iteration number =",i)
        
    #iterate
    i += 1
    
    #output some information
    s = "i = %3d\tx = %9.8f\th = %9.8f\tb=9.8f" % (i,xi, h_step, b)
    print(s)
    
    #breal if new xi is == b
    if(xi==b):
        flag = 0
        
    #return the answer
    return x,y

SyntaxError: 'return' outside function (<ipython-input-7-7f2c093b6bd9>, line 71)